In [0]:
from pyspark.sql import functions as F

FACT_TABLE = "ironman.gold.fact_race_results"
DIM_ATHLETES = "ironman.gold.dim_athletes"
DIM_DIVISIONS = "ironman.gold.dim_divisions"
DIM_COUNTRIES = "ironman.gold.dim_countries"

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_kpi_metrics AS
SELECT
COUNT(*) as total_athletes,
SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) as total_finishers,
SUM(CASE WHEN is_dnf THEN 1 ELSE 0 END) as total_dnf,
SUM(CASE WHEN is_dns THEN 1 ELSE 0 END) as total_dns,
ROUND(SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct,
ROUND(AVG(CASE WHEN is_finisher THEN finish_time_seconds END) / 3600, 2) as avg_finish_hours,
ROUND(MIN(CASE WHEN is_finisher AND rank = 1 THEN finish_time_seconds END) / 3600, 2) as fastest_finish_hours,
COUNT(DISTINCT country_key) as total_countries,
COUNT(DISTINCT year) as total_years,
MIN(year) as first_year,
MAX(year) as latest_year
FROM ironman.gold.fact_race_results;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_kpi_metrics;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_athletes_by_year AS
SELECT
year,
source_gender as gender,
COUNT(*) as total_athletes,
SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) as finishers,
SUM(CASE WHEN is_dnf THEN 1 ELSE 0 END) as dnf,
SUM(CASE WHEN is_dns THEN 1 ELSE 0 END) as dns,
ROUND(SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct
FROM ironman.gold.fact_race_results
GROUP BY year, source_gender
ORDER BY year, source_gender;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_athletes_by_year;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_finish_rate_trend AS
SELECT
year,
COUNT(*) as total_athletes,
SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) as finishers,
ROUND(SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct,
ROUND(AVG(CASE WHEN is_finisher THEN finish_time_seconds END) / 3600, 2) as avg_finish_hours
FROM ironman.gold.fact_race_results
GROUP BY year
ORDER BY year;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_finish_rate_trend;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_gender_distribution AS
SELECT
year,
CASE
WHEN source_gender = 'M' THEN 'Male'
ELSE source_gender
END as gender,
COUNT(*) as athletes,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY year), 1) as percentage
FROM ironman.gold.fact_race_results
GROUP BY year, source_gender
ORDER BY year, source_gender;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_gender_distribution;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_top_countries AS
SELECT
c.country_name,
c.continent,
COUNT(*) as total_athletes,
SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) as finishers,
ROUND(SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct,
ROUND(AVG(CASE WHEN f.is_finisher THEN f.finish_time_seconds END) / 3600, 2) as avg_finish_hours,
COUNT(DISTINCT f.year) as years_participated
FROM ironman.gold.fact_race_results f
LEFT JOIN ironman.gold.dim_countries c ON f.country_key = c.country_key
WHERE c.country_name IS NOT NULL
GROUP BY c.country_name, c.continent
ORDER BY total_athletes DESC;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_top_countries LIMIT 15;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_countries_by_year AS
SELECT
f.year,
c.country_name,
c.continent,
COUNT(*) as total_athletes,
SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) as finishers
FROM ironman.gold.fact_race_results f
LEFT JOIN ironman.gold.dim_countries c ON f.country_key = c.country_key
WHERE c.country_name IS NOT NULL
GROUP BY f.year, c.country_name, c.continent
ORDER BY f.year, total_athletes DESC;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_continent_distribution AS
SELECT
year,
c.continent,
COUNT(*) as athletes,
ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY year), 1) as percentage
FROM ironman.gold.fact_race_results f
LEFT JOIN ironman.gold.dim_countries c ON f.country_key = c.country_key
WHERE c.continent IS NOT NULL AND c.continent != 'Unknown'
GROUP BY year, c.continent
ORDER BY year, athletes DESC;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_continent_distribution;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_segment_times AS
SELECT
year,
CASE WHEN source_gender = 'M' THEN 'Male' ELSE 'Female' END as gender,
ROUND(AVG(swim_time_seconds) / 60, 1) as avg_swim_minutes,
ROUND(AVG(transition_1_seconds) / 60, 1) as avg_t1_minutes,
ROUND(AVG(bike_time_seconds) / 60, 1) as avg_bike_minutes,
ROUND(AVG(transition_2_seconds) / 60, 1) as avg_t2_minutes,
ROUND(AVG(run_time_seconds) / 60, 1) as avg_run_minutes,
ROUND(AVG(finish_time_seconds) / 60, 1) as avg_total_minutes,
-- Percentages
ROUND(AVG(swim_time_seconds) * 100.0 / AVG(finish_time_seconds), 1) as swim_pct,
ROUND(AVG(bike_time_seconds) * 100.0 / AVG(finish_time_seconds), 1) as bike_pct,
ROUND(AVG(run_time_seconds) * 100.0 / AVG(finish_time_seconds), 1) as run_pct
FROM ironman.gold.fact_race_results
WHERE is_finisher = true AND has_data_issue = false
GROUP BY year, source_gender
ORDER BY year, source_gender;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_segment_times;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_age_group_performance AS
SELECT
f.year,
d.age_group_start,
d.age_group_end,
CONCAT(d.age_group_start, '-', d.age_group_end) as age_group,
COUNT(*) as total_athletes,
SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) as finishers,
ROUND(SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct,
ROUND(AVG(CASE WHEN f.is_finisher THEN f.finish_time_seconds END) / 3600, 2) as avg_finish_hours
FROM ironman.gold.fact_race_results f
LEFT JOIN ironman.gold.dim_divisions d ON f.division_key = d.division_key
WHERE d.age_group_start IS NOT NULL AND d.is_professional = false
GROUP BY f.year, d.age_group_start, d.age_group_end
ORDER BY f.year, d.age_group_start;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_age_group_performance;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_finish_time_distribution AS
SELECT
year,
CASE WHEN source_gender = 'M' THEN 'Male' ELSE 'Female' END as gender,
FLOOR(finish_time_seconds / 3600) as finish_hour,
CONCAT(FLOOR(finish_time_seconds / 3600), '-', FLOOR(finish_time_seconds / 3600) + 1, ' hrs') as finish_hour_range,
COUNT(*) as athletes
FROM ironman.gold.fact_race_results
WHERE is_finisher = true AND finish_time_seconds IS NOT NULL
GROUP BY year, source_gender, FLOOR(finish_time_seconds / 3600)
ORDER BY year, source_gender, finish_hour;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_finish_time_distribution;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_top_finishers AS
SELECT
f.year,
f.rank,
a.athlete_name,
c.country_name,
c.continent,
d.division_description,
CASE WHEN f.source_gender = 'M' THEN 'Male' ELSE 'Female' END as gender,
f.finish_time_seconds,
CONCAT(
FLOOR(f.finish_time_seconds / 3600), ':',
LPAD(FLOOR((f.finish_time_seconds % 3600) / 60), 2, '0'), ':',
LPAD(f.finish_time_seconds % 60, 2, '0')
) as finish_time_formatted,
ROUND(f.swim_time_seconds / 60, 1) as swim_minutes,
ROUND(f.bike_time_seconds / 60, 1) as bike_minutes,
ROUND(f.run_time_seconds / 60, 1) as run_minutes
FROM ironman.gold.fact_race_results f
LEFT JOIN ironman.gold.dim_athletes a ON f.athlete_key = a.athlete_key
LEFT JOIN ironman.gold.dim_countries c ON f.country_key = c.country_key
LEFT JOIN ironman.gold.dim_divisions d ON f.division_key = d.division_key
WHERE f.is_finisher = true AND f.rank IS NOT NULL
ORDER BY f.year DESC, f.rank;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_top_finishers WHERE rank <= 10;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_year_over_year AS
WITH yearly_stats AS (
SELECT
year,
COUNT(*) as total_athletes,
SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) as finishers,
ROUND(SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct,
ROUND(AVG(CASE WHEN is_finisher THEN finish_time_seconds END) / 3600, 2) as avg_finish_hours,
COUNT(DISTINCT country_key) as countries
FROM ironman.gold.fact_race_results
GROUP BY year
)
SELECT
curr.year,
curr.total_athletes,
curr.finishers,
curr.finish_rate_pct,
curr.avg_finish_hours,
curr.countries,
-- Year over Year changes
curr.total_athletes - COALESCE(prev.total_athletes, 0) as athletes_change,
ROUND((curr.total_athletes - COALESCE(prev.total_athletes, curr.total_athletes)) * 100.0 / NULLIF(prev.total_athletes, 0), 1) as athletes_change_pct,
curr.finish_rate_pct - COALESCE(prev.finish_rate_pct, 0) as finish_rate_change,
curr.avg_finish_hours - COALESCE(prev.avg_finish_hours, 0) as avg_time_change_hours
FROM yearly_stats curr
LEFT JOIN yearly_stats prev ON curr.year = prev.year + 1
ORDER BY curr.year;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_year_over_year;


In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_pro_vs_age_group AS
SELECT
f.year,
CASE WHEN d.is_professional THEN 'Professional' ELSE 'Age Group' END as category,
COUNT(*) as athletes,
SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) as finishers,
ROUND(SUM(CASE WHEN f.is_finisher THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as finish_rate_pct,
ROUND(AVG(CASE WHEN f.is_finisher THEN f.finish_time_seconds END) / 3600, 2) as avg_finish_hours,
ROUND(MIN(CASE WHEN f.is_finisher THEN f.finish_time_seconds END) / 3600, 2) as fastest_hours
FROM ironman.gold.fact_race_results f
LEFT JOIN ironman.gold.dim_divisions d ON f.division_key = d.division_key
WHERE d.is_professional IS NOT NULL
GROUP BY f.year, d.is_professional
ORDER BY f.year, category;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_pro_vs_age_group;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_dnf_analysis AS
SELECT
year,
CASE WHEN source_gender = 'M' THEN 'Male' ELSE 'Female' END as gender,
COUNT(*) as total_athletes,
SUM(CASE WHEN is_finisher THEN 1 ELSE 0 END) as finishers,
SUM(CASE WHEN is_dnf THEN 1 ELSE 0 END) as dnf,
SUM(CASE WHEN is_dns THEN 1 ELSE 0 END) as dns,
SUM(CASE WHEN is_dq THEN 1 ELSE 0 END) as dq,
ROUND(SUM(CASE WHEN is_dnf THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as dnf_rate_pct,
ROUND(SUM(CASE WHEN is_dns THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 1) as dns_rate_pct
FROM ironman.gold.fact_race_results
GROUP BY year, source_gender
ORDER BY year, source_gender;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_dnf_analysis;

In [0]:
%sql
CREATE OR REPLACE VIEW ironman.gold.vw_fastest_times AS
SELECT
year,
CASE WHEN source_gender = 'M' THEN 'Male' ELSE 'Female' END as gender,
-- Fastest overall
ROUND(MIN(finish_time_seconds) / 3600, 2) as fastest_finish_hours,
-- Fastest segments
ROUND(MIN(swim_time_seconds) / 60, 1) as fastest_swim_minutes,
ROUND(MIN(bike_time_seconds) / 60, 1) as fastest_bike_minutes,
ROUND(MIN(run_time_seconds) / 60, 1) as fastest_run_minutes,
-- Averages
ROUND(AVG(finish_time_seconds) / 3600, 2) as avg_finish_hours,
-- Median (approximate using percentile)
ROUND(PERCENTILE_APPROX(finish_time_seconds, 0.5) / 3600, 2) as median_finish_hours
FROM ironman.gold.fact_race_results
WHERE is_finisher = true
GROUP BY year, source_gender
ORDER BY year, source_gender;

In [0]:
%sql
SELECT * FROM ironman.gold.vw_fastest_times;

In [0]:
dbutils.notebook.exit("SUCCESS")